This file is for evaluating the performance of methods on large tabular datasets.

In [1]:
!echo $HOSTNAME

amueller-4gpu-eastus2-3


In [2]:
import matplotlib.pyplot as plt

from mothernet.evaluation.baselines import tabular_baselines

import seaborn as sns
import numpy as np
import warnings
warnings.simplefilter("ignore", FutureWarning)  # openml deprecation of array return type
from mothernet.datasets import load_openml_list, open_cc_valid_dids, open_cc_dids, open_cc_large_dids
from mothernet.evaluation.baselines.tabular_baselines import knn_metric, catboost_metric, logistic_metric, xgb_metric, random_forest_metric, mlp_metric, hyperfast_metric, hyperfast_metric_tuning, resnet_metric, mothernet_init_metric
from mothernet.evaluation.tabular_evaluation import evaluate, eval_on_datasets, transformer_metric
from mothernet.evaluation import tabular_metrics
from mothernet.prediction.tabpfn import TabPFNClassifier
import os
from mothernet.evaluation.baselines.distill_mlp import DistilledTabPFNMLP
from mothernet.prediction.mothernet import MotherNetClassifier
from functools import partial
from mothernet.evaluation.tabular_evaluation import eval_on_datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from mothernet.prediction.mothernet import ShiftClassifier, EnsembleMeta, MotherNetClassifier
from sklearn.impute import SimpleImputer
from mothernet.prediction.mothernet_additive import MotherNetAdditiveClassifier

from interpret.glassbox import ExplainableBoostingClassifier


from hyperfast import HyperFastClassifier

# transformers don't have max times
import warnings
import pandas as pd

import datetime

import pickle

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

# Datasets

In [12]:
cc_test_datasets_multiclass, cc_test_datasets_multiclass_df = load_openml_list(
    [
        146818,
        9910,
        14969,
        168335,
        146607,
        3896,
        189354,
        189356,
        14964,
        7,
        11,
        9981,
        25,
        29,
        31,
        219,
        3711,
        168337,
        50,
        146606,
        168911,
        167119,
        3917,
        10,
        14,
        22,
        146065,
        9977,
        9956,
        9952,
        9890,
        3561,
        9957,
        3797,
        45,
        53
    ], 
    multiclass=True,
    shuffled=True, 
    filter_for_nan=False, 
    max_samples = 1000000000, 
    num_feats=1000000, 
    return_capped=True
)

Number of datasets: 15
Too many classes
Too many features
Too many features
Too many features
Too many features
Too many features


In [5]:
eval_positions = [50000]
max_features = 100
n_samples = 100000
base_path = os.path.join('../')
overwrite = False
# max_times only affect non-nn models, nn models are not affected by max_times
# for non-nn models, when the runtime is longer than the max_time, it should stop
max_times = [1, 15, 30, 60, 60 * 5, 60 * 15, 60*60] 
metric_used = tabular_metrics.auc_metric
task_type = 'multiclass'

In [6]:
!mkdir -p {base_path}/results
!mkdir -p {base_path}/results/tabular/
!mkdir -p {base_path}/results/tabular/multiclass/

In [7]:
cc_test_datasets_multiclass_df['isNumeric'] = (
    cc_test_datasets_multiclass_df.NumberOfSymbolicFeatures == 1
    ) & (cc_test_datasets_multiclass_df.NumberOfInstancesWithMissingValues == 0)

In [8]:
cc_test_datasets_multiclass_df['NumberOfInstances'] =  cc_test_datasets_multiclass_df['NumberOfInstances'].astype(int)
cc_test_datasets_multiclass_df['NumberOfFeatures'] =  cc_test_datasets_multiclass_df['NumberOfFeatures'].astype(int)
cc_test_datasets_multiclass_df['NumberOfClasses'] =  cc_test_datasets_multiclass_df['NumberOfClasses'].astype(int)

print(cc_test_datasets_multiclass_df[['did', 'name', 'NumberOfFeatures', 'NumberOfInstances', 'NumberOfClasses']].rename(columns={'NumberOfFeatures': "d", "NumberOfInstances":"n", "NumberOfClasses": "k"}).to_latex(index=False))

\begin{tabular}{rlrrr}
\toprule
did & name & d & n & k \\
\midrule
7 & audiology & 70 & 226 & 24 \\
10 & lymph & 19 & 148 & 4 \\
11 & balance-scale & 5 & 625 & 3 \\
14 & mfeat-fourier & 77 & 2000 & 10 \\
22 & mfeat-zernike & 48 & 2000 & 10 \\
25 & colic & 27 & 368 & 2 \\
29 & credit-approval & 16 & 690 & 2 \\
31 & credit-g & 21 & 1000 & 2 \\
50 & tic-tac-toe & 10 & 958 & 2 \\
53 & heart-statlog & 14 & 270 & 2 \\
3561 & QSAR-TID-17034 & 1026 & 116 & 0 \\
3711 & QSAR-TID-10331 & 1026 & 303 & 0 \\
3797 & QSAR-TID-11589 & 1026 & 82 & 0 \\
3896 & QSAR-TID-100824 & 1026 & 79 & 0 \\
3917 & QSAR-TID-30037 & 1026 & 711 & 0 \\
\bottomrule
\end{tabular}



In [9]:
preview = cc_test_datasets_multiclass_df[['did', 'name', 'NumberOfFeatures', 'NumberOfInstances', 'NumberOfClasses']].reset_index(drop=True)
preview

,did,name,NumberOfFeatures,NumberOfInstances,NumberOfClasses
0,7,audiology,70,226,24
1,10,lymph,19,148,4
2,11,balance-scale,5,625,3
3,14,mfeat-fourier,77,2000,10
4,22,mfeat-zernike,48,2000,10
5,25,colic,27,368,2
6,29,credit-approval,16,690,2
7,31,credit-g,21,1000,2
8,50,tic-tac-toe,10,958,2
9,53,heart-statlog,14,270,2


openml__california__361089              | Processing...


In [24]:
dataset.get_metadata()

{'name': 'openml__california__361089',
 'cat_idx': [],
 'cat_dims': [],
 'target_type': 'regression',
 'num_classes': 1,
 'num_features': 8,
 'num_instances': 20640,
 'split_source': 'openml'}